### Dissertation Data analysis

In [ ]:
# Import Libraries 
import pandas as pd
import os
import glob
import seaborn as sns
import zipfile
import re
from sentence_transformers import SentenceTransformer
import numpy as np
import torch
import datetime
from dateutil.parser import parse
pd.options.display.max_columns = None


def get_moved(x, col = 'bus_prox_class'):
    if not x['moved']:
        return False, False, False
    else:
        if pd.isna(x[f'prev_{col}']):
            return True, False, False
        else:
            if x[f'prev_{col}'] < x[col]:
                if col == 'prox_class':
                    return False, True, False
                else:
                    return True, False, False
            elif x[f'prev_{col}'] > x[col]:
                if col == 'prox_class':
                    return True, False, False
                else:
                    return False, True, False
            else:
                return False, False, True

def get_moved_within(x, col = 'oa21cd'):
    if not x['moved']:
        return False
    else:
        if pd.isna(x[f'prev_{col}']):
            return False
        else:
            if x[f'prev_{col}'] == x[col]:
                return True
            else:
                return False

def get_status_date(x, df):
    if pd.isna(x['status_date']):
        tmp = df[(df['rates_code'] == x['rates_code']) & (df['location_code'] == x['location_code'])].copy()
        if len(tmp):
            return tmp['status_date'].iloc[0]
        else:
            if pd.isna(x['occupation_date']):
                return x['from_date']
            else:
                return x['occupation_date']
    else:
        return x['status_date']

# Preprocess Business Rate Data and Clean wrong Postcodes

In [ ]:
# Upload data files 
# use glob to get all the CSV files in the folder
path_lambeth = 'Data\\business rate data\\Lambeth\\'
csv_files = []
csv_files.extend(glob.glob(os.path.join(path_lambeth, "*.csv")))

path_southwark = 'Data\\business rate data\\Southwark\\'
csv_files.extend(glob.glob(os.path.join(path_southwark, "*.csv")))

excel_files = glob.glob(os.path.join(path_southwark, "*.xlsx"))

# loop over the list of CSV files
wands_br_df= []
for f in csv_files:
    df = pd.read_csv(f)
    df['quarter']= f.split("\\")[-1][:7]
    wands_br_df.append(df)

for f in excel_files:
    df = pd.read_excel(f)
    df['quarter']= f.split("\\")[-1][:7]
    wands_br_df.append(df)
# Dataframe concatenation
wands_br_df= pd.concat(wands_br_df, ignore_index=True)

wands_br_2024_df = pd.concat([pd.read_csv("Data\\business rate data\\alswliman\\period-2021-to-2024-E09000022-lambeth.csv"),
                              pd.read_csv("Data\\business rate data\\alswliman\\period-2021-to-2024-E09000028-southwark.csv")],
                            ignore_index = True)
wands_br_2024_df = wands_br_2024_df.rename(columns = {'filter_period' : 'period', 'billing_authority_name' : 'rates_authority',
                     'geocode' : 'rates_authority_id', 'uarn' : 'location_code', 'billing_reference' : 'rates_code',
                     'account_name' : 'name', 'account_start_date' : 'status_date', 'searchable_address' : 'address_street',
                     'postcode_id' : 'pc__pcs', 'category_id' : 'scat_code', 'primary_description' : 'use_category',
                     'category_subgroup' : 'sub_category', 'category_group' : 'category', 'rates_payable' : 'rates_expected',
                     'rateable_value' : 'rental_valuation', 'total_floor_area' : 'floor_area'})
wands_br_2024_df = wands_br_2024_df.drop(columns = ['geometry', 'occupation_state', 'unit_of_measure', 'series', 'epoch'])
print(wands_br_2024_df.shape)
wands_br_2024_df['period'] = wands_br_2024_df['period'].apply(lambda x: '-'.join([x.split('-')[0], x.split('-')[1]]))
wands_br_2024_df['quarter'] = wands_br_2024_df['period']
wands_br_2024_df = wands_br_2024_df[wands_br_2024_df['period'].apply(lambda x: int(x.split('-')[1])) == 12].copy()
print(wands_br_2024_df.shape)
wands_br_2024_df['status_date'] = wands_br_2024_df.groupby(['rates_code', 'location_code'])['status_date'].ffill()
wands_br_2024_df['status_date'] = wands_br_2024_df.groupby(['rates_code', 'location_code'])['status_date'].bfill()
print(wands_br_2024_df.shape)
print(wands_br_2024_df[wands_br_2024_df['status_date'].apply(pd.isna)].shape)
wands_br_rates_code_df = wands_br_df[['rates_code', 'location_code', 'name', 'status_date'
                                     ]].rename(columns = {'status_date' : 'old_status_date'})
wands_br_rates_code_df['old_status_date'] = wands_br_rates_code_df['old_status_date'].apply(
    lambda x: str(parse(x)).split(' ')[0] if isinstance(x, str) else str(x).split(' ')[0])
wands_br_rates_code_df = wands_br_rates_code_df.drop_duplicates()
wands_br_rates_code_df = wands_br_rates_code_df.groupby(['rates_code', 'location_code', 'name']).min().reset_index()
print(wands_br_rates_code_df.shape)
print(wands_br_2024_df.shape)
wands_br_2024_df = wands_br_2024_df.set_index(['rates_code', 'location_code', 'name']).join(
    wands_br_rates_code_df.set_index(['rates_code', 'location_code', 'name'])).reset_index()
print(wands_br_2024_df.shape)
wands_br_2024_df['status_date'] = wands_br_2024_df.apply(lambda x: x['old_status_date'] if pd.isna(x['status_date'])
                                                         else x['status_date'], axis = 1)
wands_br_2024_df = wands_br_2024_df.drop(columns = ['old_status_date'])
wands_br_2024_df['status_date'] = wands_br_2024_df.apply(lambda x: x['occupation_date'] if pd.isna(x['status_date'])
                                                         else x['status_date'], axis = 1)
wands_br_2024_df['status_date'] = wands_br_2024_df.apply(lambda x: x['from_date'] if pd.isna(x['status_date'])
                                                         else x['status_date'], axis = 1)
print(wands_br_2024_df.shape)
print(wands_br_2024_df[wands_br_2024_df['status_date'].apply(pd.isna)].shape)
drop_cols = list(set(wands_br_2024_df.columns) - set(wands_br_df.columns))
print(drop_cols)
wands_br_2024_df = wands_br_2024_df.drop(columns = drop_cols)
ext_cols = list(set(wands_br_df.columns) - set(wands_br_2024_df.columns))
print(ext_cols)
for col in ext_cols:
    wands_br_2024_df[col] = np.nan

wands_br_df = pd.concat([wands_br_df, wands_br_2024_df[wands_br_df.columns]], ignore_index = True)
print(wands_br_df.shape)
print(wands_br_2024_df.shape)
print(wands_br_df.shape)
wands_br_df.head()

In [ ]:
pc_lon_lat_map_df = pd.read_csv("Data/ONSPD_MAY_2025_UK_SW.csv")
pc_lon_lat_map_df = pc_lon_lat_map_df[['pcd2', 'lat', 'long']]
pc_lon_lat_map_df['pcd2'] = pc_lon_lat_map_df['pcd2'].apply(lambda x: ' '.join([x.split(' ')[0],
                                                            x.split(' ')[-1]]))

pc_lsoa_mapping_df = pd.read_csv("Data\\PCD_OA21_LSOA21\\PCD_OA21_LSOA21.csv", encoding = "ISO-8859-1")
pc_lsoa_mapping_df = pc_lsoa_mapping_df[(pc_lsoa_mapping_df['ladnm'] == 'Lambeth') |
                                        (pc_lsoa_mapping_df['ladnm'] == 'Southwark')].copy()
pc_lsoa_mapping_df.head()

In [ ]:
closest_pc_mapping = {'SW12 0BP' : 'SW12 0BT', 'SW12 0AB' : 'SW12 0BT',
'SW12 0HY' : 'SW12 0PJ', 'SW12 0NA' : 'SW12 0BS', 'SW12 0PH' : 'SW12 0PJ', 'SW12 0LT' : 'SW12 0BT',
'SW12 0PQ' : 'SW12 0PJ', 'SW12 0PS' : 'SW12 0PL', 'SW17 0AY' : 'SW17 0AS',
'SW17 0BH' : 'SW17 0BQ', 'SW4 0PT' : 'SW4 0QE', 'SW8 3LG' : 'SW8 3DJ'}
wands_br_df['closest_pc'] = wands_br_df['pc__pcs'].apply(lambda x: x if x not in closest_pc_mapping
                                                        else closest_pc_mapping[x])
wands_br_lsoa_df = wands_br_df.set_index('closest_pc').join(pc_lsoa_mapping_df.rename(
    columns = {'pcds' : 'closest_pc'}).set_index('closest_pc')[['oa21cd', 'lsoa21cd', 'msoa21cd']]).reset_index()
wands_br_lsoa_df['year'] = wands_br_lsoa_df['quarter'].apply(lambda x: int(x.split('-')[0]))
wands_br_lsoa_df['address_no'] = wands_br_lsoa_df['address_no'].fillna('')
wands_br_lsoa_df['address_street'] = wands_br_lsoa_df['address_street'].fillna('')
wands_br_lsoa_df['addressline1'] = wands_br_lsoa_df.apply(lambda x:
                                  ' '.join([str(x['address_no']), x['address_street']]), axis = 1)
wands_br_lsoa_df['addressline1_merge'] = wands_br_lsoa_df['addressline1'].fillna('').apply(
    lambda x: re.sub(r'[^\w\s]','',x.lower()))

#fix missing names
br_cols = ['closest_pc', 'pc__pcs', 'rates_code', 'location_code', 'address_no',
           'address_street', 'use_category', 'sub_category', 'category', 'rental_valuation',
           'valuation_date', 'period', 'rates_expected', 'name', 'status', 'status_date', 'quarter']
wands_br_lsoa_df = wands_br_lsoa_df.sort_values(by = ['rates_code', 'period'])
wands_br_lsoa_df['name'] = wands_br_lsoa_df['name'].bfill()

#add min and mex periods
wands_br_lsoa_df = wands_br_lsoa_df.set_index('rates_code').join(wands_br_lsoa_df[['rates_code', 'period'
        ]].groupby('rates_code').min().rename(columns = {'period' : 'min_period'})).reset_index()
wands_br_lsoa_df = wands_br_lsoa_df.set_index('rates_code').join(wands_br_lsoa_df[['rates_code', 'period'
        ]].groupby('rates_code').max().rename(columns = {'period' : 'max_period'})).reset_index()

#reduce business rate data to yearly
wands_br_lsoa_df['quarter'] = wands_br_lsoa_df['quarter'].apply(lambda x: int(x.split('-')[1]))
quarter_wands_br_lsoa_df = wands_br_lsoa_df.copy()
wands_br_lsoa_df = wands_br_lsoa_df[(wands_br_lsoa_df['quarter'] == 12)].copy()

#map post code to lon, lat
wands_br_lsoa_df = wands_br_lsoa_df.set_index('closest_pc').join(pc_lon_lat_map_df.rename(columns = {
    'pcd2' : 'closest_pc'}).set_index('closest_pc')).reset_index()

# add dissolutiondate and incorporationdate
wands_br_lsoa_df['dissolutiondate'] = wands_br_lsoa_df['max_period'].apply(lambda x: np.nan
                                                   if x == '2024-12' else x + '-01')
wands_br_lsoa_df['incorporationdate'] = wands_br_lsoa_df['status_date']


#identify moved businesses
wands_br_lsoa_df['postcode'] = wands_br_lsoa_df['closest_pc']
wands_br_lsoa_df = wands_br_lsoa_df.sort_values(by = ['rates_code', 'year'])
wands_br_lsoa_df['prev_postcode'] = wands_br_lsoa_df.groupby('rates_code')['postcode'].shift()
wands_br_lsoa_df['moved'] = wands_br_lsoa_df.apply(lambda x:
        ((x['prev_postcode'] != x['postcode']) and (not pd.isna(x['prev_postcode']))), axis = 1)

#add company name and company id
wands_br_lsoa_df['companyname'] = wands_br_lsoa_df['name']
wands_br_lsoa_df['rates_code'] = wands_br_lsoa_df['rates_code'].apply(lambda x: str(int(float(x))) if 'E+' in str(x) else x)
wands_br_lsoa_df['CompanyID'] = wands_br_lsoa_df['rates_code']

#########################
print(wands_br_lsoa_df.shape)
t = wands_br_lsoa_df.fillna('').groupby(['year', 'CompanyID']).count().sort_values(by = 'companyname').reset_index()
ids = t[t['companyname'] > 1]['CompanyID'].unique()
wands_br_lsoa_df_tmp = wands_br_lsoa_df[wands_br_lsoa_df['CompanyID'].isin(ids)].copy()
wands_br_lsoa_df_tmp['CompanyID'] = wands_br_lsoa_df_tmp.apply(lambda x: '-'.join([x['CompanyID'],
                                                                       str(x['location_code']), x['name']]), axis = 1)
wands_br_lsoa_df = pd.concat([wands_br_lsoa_df_tmp, wands_br_lsoa_df[~wands_br_lsoa_df['CompanyID'].isin(ids)]])
print(wands_br_lsoa_df.shape)
#########################

#add scat categories and survived
wands_br_lsoa_df['category'] = wands_br_lsoa_df.apply(lambda x: x['sub_category']
                                  if pd.isna(x['category']) else x['category'], axis = 1)
wands_br_lsoa_df['scat_sub_category'] = wands_br_lsoa_df['sub_category']
wands_br_lsoa_df['scat_category'] = wands_br_lsoa_df['category']
wands_br_lsoa_df['survived'] = wands_br_lsoa_df['dissolutiondate'].apply(lambda x: pd.isna(x))

wands_br_lsoa_df = wands_br_lsoa_df.rename(columns = {'rates_authority' : 'ladnm'})

wands_br_lsoa_df.head()

In [ ]:
s = wands_br_lsoa_df.groupby(['CompanyID', 'companyname', 'year']).count()[['postcode']
                    ].reset_index().groupby(['companyname', 'year']).count().reset_index()
print(s[s['CompanyID'] > 1].shape)
print(s[s['CompanyID'] == 1].shape)
companies = s['companyname'].unique()
non_uniq_companies = s[s['CompanyID'] != 1]['companyname'].unique()
uniq_companies = list(set(companies) - set(non_uniq_companies))
print(len(uniq_companies))
print(len(non_uniq_companies))
print(len(companies))
print(wands_br_lsoa_df.shape)
wands_br_lsoa_df = wands_br_lsoa_df[wands_br_lsoa_df['companyname'].isin(uniq_companies)]
print(wands_br_lsoa_df.shape)

### Processing Business Census Data 

In [ ]:
## Load Wandsworth ONS mapping data

cols = ['CompanyID', 'CompanyNumber', 'CompanyName', 'CompanyStatus', 'PostCode',
       'oa11', 'lsoa11', 'msoa11', 'oa21', 'lsoa21', 'msoa21', 'PostCodeLatitude', 'PostCodeLongitude']
ons_df = pd.read_csv("Data\Business Cencus CDRC\entities\\ONSGeography.csv.zip",
                      compression='zip', usecols=cols)
wands_ons_df = ons_df[ons_df['lsoa21'].isin(pc_lsoa_mapping_df['lsoa21cd'].unique())].copy()
wands_ons_df.to_csv('Data\Business Cencus CDRC\entities\lam_south_ONSGeography.csv', index = False)

## Combine Business Census data with Wandsworth ONS data

wands_year_census_df = []
for i in range(13):
    year = 2012 + i
    census_df = pd.read_csv(f"Data\\Business Cencus CDRC\\business_census{year}.csv.zip", compression='zip')
    census_df['data_year'] = year
    tmp = census_df.rename(columns = {'id' : 'CompanyID'}).set_index('CompanyID').join(
        wands_ons_df.set_index('CompanyID'), how = 'inner')
    print(tmp.shape)
    wands_year_census_df.append(tmp)
wands_year_census_df = pd.concat(wands_year_census_df).reset_index()

## Added code to cleanup dataframe
ons_cols = ['CompanyNumber', 'CompanyName', 'CompanyStatus', 'PostCode']
wands_year_census_df = wands_year_census_df.rename(columns = {col:f'ons_{col.lower()}' for col in ons_cols})
wands_year_census_df = wands_year_census_df.rename(columns = {'PostCodeLatitude' : 'lat',
                                                             'PostCodeLongitude' : 'lon'})
wands_year_census_df['addressline1'] = wands_year_census_df['addressline1'].fillna('')
wands_year_census_df.to_csv('Data\\Business Cencus CDRC\\lam_south_business_census_lsoa.csv', index = False)


##clean post code data

wands_year_census_df = wands_year_census_df.sort_values(by = ['CompanyID', 'data_year'])
#handle post code length 4 and 3 and 10 and 12
wands_year_census_df = wands_year_census_df[wands_year_census_df['postcode'] != 'KENT'].copy()
wands_year_census_df['postcode'] = wands_year_census_df['postcode'].apply(lambda x: x if pd.isna(x) else x.strip())

wands_year_census_df['postcode'] = wands_year_census_df['postcode'].apply(lambda x: 'SW16 3SN' if
                                                      x == '.' else x)
wands_year_census_df['postcode'] = wands_year_census_df['postcode'].apply(lambda x: 'N7 9BH' if
                                                      x == 'N7' else x)
wands_year_census_df['postcode'] = wands_year_census_df.apply(lambda x: 'SE1 0LH' if
          (x['postcode'] == 'SE1') and (x['CompanyID'] == 2154406) else x['postcode'], axis = 1)
wands_year_census_df['postcode'] = wands_year_census_df.apply(lambda x: 'SE1 4QF' if
          (x['postcode'] == 'SE1') and (x['CompanyID'] == 4352377) else x['postcode'], axis = 1)
wands_year_census_df['postcode'] = wands_year_census_df.apply(lambda x: 'SE1 7NA' if
          (x['postcode'] == 'SE1') and (x['CompanyID'] == 2332166) else x['postcode'], axis = 1)
wands_year_census_df['postcode'] = wands_year_census_df.apply(lambda x: 'SE1 5JH' if
          (x['postcode'] == 'SE1') and (x['CompanyID'] == 498298) else x['postcode'], axis = 1)
wands_year_census_df['postcode'] = wands_year_census_df.apply(lambda x: 'SW4 6JF' if
          (x['postcode'] == 'SW4') and (x['CompanyID'] == 37018) else x['postcode'], axis = 1)
wands_year_census_df['postcode'] = wands_year_census_df.apply(lambda x: 'SW2 1EW' if
          (x['postcode'] == 'SW2') and (x['CompanyID'] == 30721) else x['postcode'], axis = 1)
wands_year_census_df['postcode'] = wands_year_census_df['postcode'].apply(lambda x: 'TN11 0BS' if
                                                      x == 'TN11' else x)
wands_year_census_df['postcode'] = wands_year_census_df['postcode'].apply(lambda x: 'SE24 0BG' if
                                                      x == 'SE24' else x)
wands_year_census_df['postcode'] = wands_year_census_df['postcode'].apply(lambda x: 'SE24 0BG' if
                                                      x == 'SE24' else x)
wands_year_census_df['postcode'] = wands_year_census_df['postcode'].apply(lambda x: 'EC2A 4HJ' if
                                                      x == 'EC2 A 4HJ' else x)
wands_year_census_df['postcode'] = wands_year_census_df['postcode'].apply(lambda x: 'SE15 1RW' if
                                                      x == 'SE 15 1RW' else x)
wands_year_census_df['postcode'] = wands_year_census_df['postcode'].apply(lambda x: 'SW11 1HA' if
                                                      x == 'SW11 1 HA' else x)
wands_year_census_df['postcode'] = wands_year_census_df['postcode'].apply(lambda x: 'SE15 1SG' if
                                                      x == 'SE15 1 SG' else x)
wands_year_census_df['postcode'] = wands_year_census_df['postcode'].apply(lambda x: 'SE28 0LD' if
                                                      x == 'SEE28 0LD' else x)
wands_year_census_df['postcode'] = wands_year_census_df['postcode'].apply(lambda x: 'SE17 3BA' if
                                                      x == 'SE 17 3BA' else x)
wands_year_census_df['postcode'] = wands_year_census_df['postcode'].apply(lambda x: 'WE17 1DS' if
                                                      x == 'WE 17 1DS' else x)
wands_year_census_df['postcode'] = wands_year_census_df['postcode'].apply(lambda x: 'SE16 4DG' if
                                                      x == 'SE16 4 DG' else x)
wands_year_census_df['postcode'] = wands_year_census_df['postcode'].apply(lambda x: 'SE15 6TQ' if
                                                      x == 'S E 15 6TQ' else x)
wands_year_census_df['postcode'] = wands_year_census_df['postcode'].apply(lambda x: 'SE21 7HP' if
                                                      x == 'SE 21 7 HP' else x)
wands_year_census_df['postcode'] = wands_year_census_df['postcode'].apply(lambda x: 'IG11 8LL' if
                                                      x == '07891312648' else x)
wands_year_census_df['postcode'] = wands_year_census_df['postcode'].apply(lambda x: 'EC4A 1BR' if
                                                      x == 'ONE EC4A 1BR' else x)
wands_year_census_df['postcode'] = wands_year_census_df['postcode'].apply(lambda x: 'SE26 6SH' if
                                                      x == 'BR3 4RHSE26 6SH' else x)

#Handle na in postcode
wands_year_census_df['postcode'] = wands_year_census_df['postcode'].bfill()
#Handle missing spaces in post code
wands_year_census_df['postcode'] = wands_year_census_df['postcode'].apply(lambda x: x if len(x) != 5
                                                     else ' '.join([x[:2], x[2:]]))

#Handle 7 char post code with no space
wands_year_census_df['postcode'] = wands_year_census_df['postcode'].apply(lambda x: ' '.join([x[:4], x[4:]])
                                                          if (len(x) == 7) and (' ' not in x) else x)
#Handle 6 char post code with no space
wands_year_census_df['postcode'] = wands_year_census_df['postcode'].apply(lambda x: ' '.join([x[:3], x[3:]])
                                                      if (len(x) == 6) and (' ' not in x) else x)
#Handle double spaces in post code
wands_year_census_df['postcode'] = wands_year_census_df['postcode'].apply(lambda x: x if pd.isna(x) else
                                     ' '.join([x.split(' ')[0], x.split(' ')[-1]]))

#clean extra chars
wands_year_census_df['postcode'] = wands_year_census_df['postcode'].apply(lambda x: x if len(x) != 9
                                                     else x[:8])

wands_year_census_df = wands_year_census_df.set_index('postcode').join(pc_lsoa_mapping_df.rename(
    columns = {'pcds' : 'postcode'}).set_index('postcode')[['oa21cd', 'lsoa21cd', 'msoa21cd',
                                                            'ladnm']]).reset_index()
wands_year_census_df.to_csv('Data\\Business Cencus CDRC\\lam_south_business_census_lsoa_pc_cleaned.csv',
                            index = False)

In [ ]:
wands_year_census_df = pd.read_csv('Data\\Business Cencus CDRC\\lam_south_business_census_lsoa_pc_cleaned.csv')
wands_year_census_df['addressline1_merge'] = wands_year_census_df['addressline1'].fillna('').apply(
    lambda x: re.sub(r'[^\w\s]','',x.lower()))
print(wands_year_census_df.shape)
print(wands_year_census_df[wands_year_census_df['lsoa21cd'].apply(pd.isna)].shape)
print(wands_year_census_df[~wands_year_census_df['lsoa21cd'].apply(pd.isna)].shape)
print(wands_year_census_df.groupby('CompanyID').count().shape)
print(wands_year_census_df.groupby(['CompanyID', 'postcode']).count().shape)
wands_year_census_df = wands_year_census_df.rename(columns = {'data_year' : 'year'})

cols = ['CompanyID', 'companynumber', 'companyname', 'addressline1', 'addressline2', 'ons_postcode',
        'oa21', 'lsoa21', 'postcode', 'oa21cd', 'lsoa21cd', 'ladnm', 'year']

#map census industry to business rate industry
sic_cat_map_df = pd.read_excel("Data/sic_cat_mapping.xlsx", sheet_name = 'sic_cat_mapping')
sic_scat_map_df = pd.read_excel("Data/sic_cat_mapping.xlsx", sheet_name = 'scat_sic_mapping')
wands_year_census_df = wands_year_census_df.sort_values(by = ['CompanyID', 'year'])
wands_year_census_df['siccode'] = wands_year_census_df['siccode'].bfill()
wands_year_census_df = wands_year_census_df.set_index('siccode').join(sic_cat_map_df.set_index('siccode')[[
    'sic_category']]).reset_index()
wands_year_census_df = wands_year_census_df.set_index('sic_category').join(sic_scat_map_df.set_index(
    'sic_category')).reset_index()
wands_year_census_df['sic_category'] = wands_year_census_df['sic_category'].fillna('Unclassified')
wands_year_census_df['scat_category'] = wands_year_census_df['scat_category'].fillna('Unclassified')
wands_year_census_df['scat_sub_category'] = wands_year_census_df['scat_sub_category'].fillna('Unclassified')

#identify moved businesses
wands_year_census_df = wands_year_census_df.sort_values(by = ['CompanyID', 'year'])
wands_year_census_df['prev_postcode'] = wands_year_census_df.groupby('CompanyID')['postcode'].shift()
wands_year_census_df['moved'] = wands_year_census_df.apply(lambda x:
        ((x['prev_postcode'] != x['postcode']) and (not pd.isna(x['prev_postcode']))), axis = 1)

# filter on wondsworth only
wands_year_census_df = wands_year_census_df[wands_year_census_df['ladnm'].isin(['Wandsworth', 'Lambeth', 'Southwark'])].copy()

#add correct lon, lat
wands_year_census_df = wands_year_census_df.rename(columns = {'lon' : 'ons_lon',
                                                              'lat' : 'ons_lat'})
wands_year_census_df = wands_year_census_df.set_index('postcode').join(pc_lon_lat_map_df.rename(columns = {
    'pcd2' : 'postcode'}).set_index('postcode')).reset_index()

#add survived column
wands_year_census_df['survived'] = wands_year_census_df['dissolutiondate'].apply(lambda x: pd.isna(x))

#add missing fields
wands_year_census_df['floor_area'] = None
wands_year_census_df['rental_valuation'] = None
wands_year_census_df['rates_expected'] = None

wands_year_census_df.head()

In [ ]:
final_cols = ['CompanyID', 'companyname', 'addressline1', 'postcode', 'dissolutiondate',
              'incorporationdate', 'survived', 'year', 'oa21cd', 'lsoa21cd', 'msoa21cd', 'ladnm', 'long', 'lat',
              'addressline1_merge', 'scat_sub_category', 'scat_category', 'prev_postcode', 'moved',
              'floor_area', 'rental_valuation', 'rates_expected']

# Match Business Rates and Business Census Datasets

In [ ]:
## get exact matches

print(wands_br_lsoa_df.shape)
print(wands_year_census_df.shape)
group_cols = ['postcode', 'addressline1_merge', 'year']
exact_match_br_census_df = wands_br_lsoa_df[final_cols].set_index(group_cols
           ).join(wands_year_census_df[final_cols].set_index(group_cols
           ), lsuffix = '_br', rsuffix = '_census', how = 'inner').reset_index()
grouped_cols_df = exact_match_br_census_df[group_cols].drop_duplicates()
grouped_cols_df['exist'] = True

print(exact_match_br_census_df.shape)
print(grouped_cols_df.shape)

no_exact_wands_br_lsoa_df = wands_br_lsoa_df.set_index(group_cols).join(grouped_cols_df.set_index(group_cols))
print(no_exact_wands_br_lsoa_df[~no_exact_wands_br_lsoa_df['exist'].apply(pd.isna)].shape)
no_exact_wands_br_lsoa_df = no_exact_wands_br_lsoa_df[no_exact_wands_br_lsoa_df['exist'].apply(pd.isna)
                            ].reset_index().drop(columns = ['exist'])
print(no_exact_wands_br_lsoa_df.shape)


no_exact_wands_year_census_df = wands_year_census_df.set_index(group_cols).join(
                    grouped_cols_df.set_index(group_cols))
print(no_exact_wands_year_census_df[~no_exact_wands_year_census_df['exist'].apply(pd.isna)].shape)
no_exact_wands_year_census_df = no_exact_wands_year_census_df[no_exact_wands_year_census_df['exist'].apply(pd.isna)
                            ].reset_index().drop(columns = ['exist'])
print(no_exact_wands_year_census_df.shape)

### get matches based on postcode and year

sub_group_cols = ['postcode', 'year']
print(no_exact_wands_br_lsoa_df.shape)
print(no_exact_wands_year_census_df.shape)
no_exact_match_br_census_df = no_exact_wands_br_lsoa_df[final_cols].set_index(sub_group_cols
           ).join(no_exact_wands_year_census_df[final_cols].set_index(sub_group_cols
           ), lsuffix = '_br', rsuffix = '_census', how = 'outer').reset_index()
print(no_exact_match_br_census_df.shape)
print(no_exact_match_br_census_df[no_exact_match_br_census_df['CompanyID_br'].apply(pd.isna)].shape)
print(no_exact_match_br_census_df[no_exact_match_br_census_df['CompanyID_census'].apply(pd.isna)].shape)

### get no matches at all and clean matches based on postcode and year

no_exact_match_br_census_df_census1 = no_exact_match_br_census_df[no_exact_match_br_census_df
                            ['CompanyID_br'].apply(pd.isna)].copy()
drop_cols = [c for c in no_exact_match_br_census_df_census1.columns if '_br' in c]
no_exact_match_br_census_df_census1 = no_exact_match_br_census_df_census1.drop(columns = drop_cols)
no_exact_match_br_census_df_census1 = no_exact_match_br_census_df_census1.rename(columns = {
    c:c.replace('_census', '') for c in no_exact_match_br_census_df_census1.columns})
no_exact_match_br_census_df_census1['source'] = 'Census'

no_exact_match_br_census_df_br1 = no_exact_match_br_census_df[no_exact_match_br_census_df
                            ['CompanyID_census'].apply(pd.isna)].copy()
drop_cols = [c for c in no_exact_match_br_census_df_br1.columns if '_census' in c]
no_exact_match_br_census_df_br1 = no_exact_match_br_census_df_br1.drop(columns = drop_cols)
no_exact_match_br_census_df_br1 = no_exact_match_br_census_df_br1.rename(columns = {
    c:c.replace('_br', '') for c in no_exact_match_br_census_df_br1.columns})
no_exact_match_br_census_df_br1['source'] = 'BR'

no_exact_match_br_census_df = no_exact_match_br_census_df[
                            (~no_exact_match_br_census_df['CompanyID_br'].apply(pd.isna)) &
                            (~no_exact_match_br_census_df['CompanyID_census'].apply(pd.isna))]

print(no_exact_match_br_census_df_census1.shape)
print(no_exact_match_br_census_df_br1.shape)
print(no_exact_match_br_census_df.shape)

# Calculate Similarity Based on Deep Neural Net Model

In [ ]:
model = SentenceTransformer('Lajavaness/bilingual-embedding-small', trust_remote_code=True)

print(wands_br_lsoa_df.shape)
print(wands_year_census_df.shape)
pc_test = wands_br_lsoa_df.set_index(['postcode',
       'year']).join(wands_year_census_df[
        cols + ['addressline1_merge']].set_index(['postcode', 'year']), rsuffix = '_census').reset_index()
print(pc_test.shape)
print(pc_test[pc_test['CompanyID'].apply(pd.isna)].shape)
print(pc_test[~pc_test['CompanyID'].apply(pd.isna)].shape)
pc_test['match'] = pc_test.apply(lambda x: x['addressline1_merge'] == x['addressline1_merge_census'], axis = 1)
print(pc_test[pc_test['match']].shape)
print(pc_test[~pc_test['match']].shape)
pc_test['addressline1'] = pc_test['addressline1'].fillna('')
pc_test['addressline1_census'] = pc_test['addressline1_census'].fillna('')

pc_test_tmp = pc_test[['postcode', 'addressline1',
      'addressline1_merge', 'addressline1_census', 'addressline1_merge_census']].drop_duplicates()

step = 10000
n = int(np.ceil(len(pc_test_tmp) / step))
pc_test_sim_df = []
print('Total steps is:', n)
for i in range(n):
    print(i)
    print(datetime.datetime.now())
    tmp = pc_test_tmp.iloc[i * step: (i+1) * step].copy()
    addressline1_embed = model.encode(tmp['addressline1'].tolist())
    addressline1_census_embed = model.encode(tmp['addressline1_census'].tolist())
    similarity = model.similarity(addressline1_embed, addressline1_census_embed)
    tmp['similarity'] = torch.diagonal(similarity, dim1=-2, dim2=-1)
    pc_test_sim_df.append(tmp)
    print(tmp.shape)
    if i % 10 == 0:
        pd.concat(pc_test_sim_df, ignore_index = True).to_pickle("lam_south_pc_test_sim_df_dedup.pkl")
pc_test_sim_df = pd.concat(pc_test_sim_df, ignore_index = True)
pc_test_sim_df.to_pickle("lam_south_pc_test_sim_df_dedup.pkl")

In [ ]:
pc_test_sim_df_dedup_final = pd.read_pickle('lam_south_pc_test_sim_df_dedup.pkl')
print(pc_test_sim_df_dedup_final.shape)
print(no_exact_match_br_census_df.shape)
test = no_exact_match_br_census_df.set_index(['postcode', 'addressline1_br', 'addressline1_census']
        ).join(pc_test_sim_df_dedup_final.rename(columns = {'addressline1' : 'addressline1_br'}
        ).set_index(['postcode', 'addressline1_br', 'addressline1_census'])[['similarity']]).reset_index()
test['similarity'] = test['similarity'].fillna(0)
print(test.shape)
print(test[test['similarity'].apply(pd.isna)].shape)
print(test[~test['similarity'].apply(pd.isna)].shape)

#calc names similarities
company_names_map_df = test[test['similarity'] >= 0.8][['postcode', 'addressline1_br', 'addressline1_census',
                   'CompanyID_br', 'CompanyID_census', 'year', 'companyname_br', 'companyname_census',
       'similarity']].sort_values(by = 'similarity')[[
    'companyname_br', 'companyname_census']].drop_duplicates()
print(company_names_map_df.shape)
company_names_map_df = pd.concat([company_names_map_df, exact_match_br_census_df[[
    'companyname_br', 'companyname_census']].drop_duplicates()])
print(company_names_map_df.shape)


step = 10000
n = int(np.ceil(len(company_names_map_df) / step))
comp_names_sim_df = []
print('Total steps is:', n)
for i in range(n):
    print(i)
    print(datetime.datetime.now())
    tmp = company_names_map_df.iloc[i * step: (i+1) * step].copy()
    companyname_br_embed = model.encode(tmp['companyname_br'].fillna('').tolist())
    companyname_census_embed = model.encode(tmp['companyname_census'].fillna('').tolist())
    similarity = model.similarity(companyname_br_embed, companyname_census_embed)
    tmp['similarity'] = torch.diagonal(similarity, dim1=-2, dim2=-1)
    comp_names_sim_df.append(tmp)
    print(tmp.shape)
    if i % 10 == 0:
        pd.concat(comp_names_sim_df, ignore_index = True).to_pickle("lam_south_comp_names_sim_df.pkl")
comp_names_sim_df = pd.concat(comp_names_sim_df, ignore_index = True)
comp_names_sim_df.to_pickle("lam_south_comp_names_sim_df.pkl")

comp_names_sim_df = pd.read_pickle('lam_south_comp_names_sim_df.pkl')
comp_names_sim_df['companyname_br'] = comp_names_sim_df['companyname_br'].fillna('')
comp_names_sim_df['companyname_census'] = comp_names_sim_df['companyname_census'].fillna('')

comp_names_sim_df = comp_names_sim_df.rename(columns = {'similarity' : 'names_similarity'})
comp_names_sim_df = comp_names_sim_df.drop_duplicates(subset = ['companyname_br', 'companyname_census'])

#merge simlarities back to no_exact_match_br_census_df
print(no_exact_match_br_census_df.shape)
no_exact_match_br_census_df = no_exact_match_br_census_df.set_index(['companyname_br', 'companyname_census']).join(
    comp_names_sim_df.set_index(['companyname_br', 'companyname_census'])).reset_index()
print(no_exact_match_br_census_df.shape)

no_exact_match_br_census_df = no_exact_match_br_census_df.set_index(['companyname_br','year']
           ).join(no_exact_match_br_census_df[['companyname_br', 'year', 'names_similarity']].groupby(
    ['companyname_br', 'year']).max().rename(columns = {'names_similarity' : 'names_max_similarity'}
                                        )).reset_index()
print(no_exact_match_br_census_df.shape)
no_exact_match_br_census_df['names_match'] = no_exact_match_br_census_df.apply(lambda x: True
       if (x['names_similarity'] == x['names_max_similarity']) and
       (x['names_similarity'] >= 0.85) else False, axis = 1)


no_exact_match_br_census_df['names_similarity'] = no_exact_match_br_census_df['names_similarity'].fillna(0)
no_exact_match_br_census_df['names_max_similarity'] = no_exact_match_br_census_df['names_max_similarity'].fillna(0)
no_exact_match_br_census_df['names_match'] = no_exact_match_br_census_df['names_match'].fillna(False)

#extract matched abd unmatched businesses
br_match_agg = no_exact_match_br_census_df[['postcode', 'year', 'companyname_br', 'CompanyID_br',
         'names_match']].groupby(['postcode', 'year', 'companyname_br', 'CompanyID_br']
                ).agg(list)['names_match'].apply(any)

census_match_agg = no_exact_match_br_census_df[['postcode', 'year', 'companyname_census', 'CompanyID_census',
         'names_match']].groupby(['postcode', 'year', 'companyname_census', 'CompanyID_census']
                ).agg(list)['names_match'].apply(any)
no_exact_match_br_census_df = no_exact_match_br_census_df.set_index(['postcode', 'year', 'companyname_br',
         'CompanyID_br']).join(br_match_agg, rsuffix = '_br').reset_index().set_index(['postcode', 'year',
         'companyname_census', 'CompanyID_census']).join(census_match_agg, rsuffix = '_census').reset_index()

# Merge Datasets Based on Matched ant Non-Matched Scnearios

In [ ]:
no_exact_match_br_census_df_match = no_exact_match_br_census_df[no_exact_match_br_census_df['names_match']].copy()
br_cols = [c for c in no_exact_match_br_census_df_match.columns if ('_br' in c) and (c != 'comb_address_no_br')] + ['CompanyID_census', 'companyname_census', 'postcode', 'year']
no_exact_match_br_census_df_match = no_exact_match_br_census_df_match[br_cols].drop_duplicates()
no_exact_match_br_census_df_match.columns = [c.replace('_br','').replace('_census','_alt')
                                         for c in no_exact_match_br_census_df_match.columns]
no_exact_match_br_census_df_match['source'] = 'Both'

no_exact_match_br_census_df_br2 = no_exact_match_br_census_df[
                        no_exact_match_br_census_df['names_match_br'] == False]
br_cols = [c for c in no_exact_match_br_census_df_br2.columns if ('_br' in c and c != 'comb_address_no_br')] + ['postcode', 'year']
no_exact_match_br_census_df_br2 = no_exact_match_br_census_df_br2[br_cols].drop_duplicates()
no_exact_match_br_census_df_br2.columns = [c.replace('_br','') for c in no_exact_match_br_census_df_br2.columns]
no_exact_match_br_census_df_br2['source'] = 'BR'

no_exact_match_br_census_df_census2 = no_exact_match_br_census_df[no_exact_match_br_census_df['names_match_census'] == False]
census_cols = [c for c in no_exact_match_br_census_df_census2.columns if ('_census' in c and c !=
                                         'comb_address_no_census')] + ['postcode', 'year']
no_exact_match_br_census_df_census2 = no_exact_match_br_census_df_census2[census_cols].drop_duplicates()
no_exact_match_br_census_df_census2.columns = [c.replace('_census','') for c in
                                              no_exact_match_br_census_df_census2.columns]
no_exact_match_br_census_df_census2['source'] = 'Census'

print(no_exact_match_br_census_df_match.shape)
print(no_exact_match_br_census_df_br2.shape)
print(no_exact_match_br_census_df_census2.shape)

In [ ]:
#exact_match_br_census_df.head()
print(exact_match_br_census_df.shape)
exact_match_br_census_df = exact_match_br_census_df.set_index(['companyname_br', 'companyname_census']).join(
    comp_names_sim_df.set_index(['companyname_br', 'companyname_census'])).reset_index()
print(exact_match_br_census_df.shape)
print(exact_match_br_census_df[exact_match_br_census_df['names_similarity'].apply(pd.isna)].shape)
print(exact_match_br_census_df[~exact_match_br_census_df['names_similarity'].apply(pd.isna)].shape)

exact_match_br_census_df = exact_match_br_census_df.set_index(['companyname_br','year']
           ).join(exact_match_br_census_df[['companyname_br', 'year', 'names_similarity']].groupby(
    ['companyname_br', 'year']).max().rename(columns = {'names_similarity' : 'names_max_similarity'}
                                        )).reset_index()
print(exact_match_br_census_df.shape)
exact_match_br_census_df['names_match'] = exact_match_br_census_df.apply(lambda x: True
       if (x['names_similarity'] == x['names_max_similarity']) and
       (x['names_similarity'] >= 0.85) else False, axis = 1)

br_cols = [c for c in exact_match_br_census_df.columns if ('_br' in c) and (c != 'comb_address_no_br')
          ] + ['CompanyID_census', 'companyname_census', 'postcode', 'year', 'addressline1_merge']

exact_match_br_census_df1 = exact_match_br_census_df[exact_match_br_census_df['names_match']].copy()
exact_match_br_census_df1 = exact_match_br_census_df1[br_cols].drop_duplicates()
exact_match_br_census_df1.columns = [c.replace('_br','').replace('_census','_alt') for c in
                                     exact_match_br_census_df1.columns]
exact_match_br_census_df1['source'] = 'Both'
print(exact_match_br_census_df1.shape)

br_cols = [c for c in exact_match_br_census_df.columns if ('_br' in c and c != 'comb_address_no_br')
          ] + ['postcode', 'year', 'addressline1_merge']

exact_match_br_census_df2 = exact_match_br_census_df[
                        exact_match_br_census_df['names_match'] == False]
exact_match_br_census_df2 = exact_match_br_census_df2[br_cols].drop_duplicates()
exact_match_br_census_df2.columns = [c.replace('_br','') for c in exact_match_br_census_df2.columns]
exact_match_br_census_df2['source'] = 'BR'
print(exact_match_br_census_df2.shape)

census_cols = [c for c in exact_match_br_census_df.columns if ('_census' in c and c != 'comb_address_no_census')
          ] + ['postcode', 'year', 'addressline1_merge']

exact_match_br_census_df3 = exact_match_br_census_df[
                        exact_match_br_census_df['names_match'] == False]
exact_match_br_census_df3 = exact_match_br_census_df3[census_cols].drop_duplicates()
exact_match_br_census_df3.columns = [c.replace('_census','') for c in exact_match_br_census_df3.columns]
exact_match_br_census_df3['source'] = 'Census'
print(exact_match_br_census_df3.shape)

In [ ]:
final_cols = final_cols + ['source']
final_br_census_df = pd.concat([no_exact_match_br_census_df_census1[final_cols],
                           no_exact_match_br_census_df_br1[final_cols],
                           no_exact_match_br_census_df_match[final_cols  + ['CompanyID_alt', 'companyname_alt']],
                           no_exact_match_br_census_df_br2[final_cols],
                           no_exact_match_br_census_df_census2[final_cols],
                           exact_match_br_census_df1[final_cols + ['CompanyID_alt', 'companyname_alt']],
                           exact_match_br_census_df2[final_cols],
                           exact_match_br_census_df3[final_cols]], ignore_index = True)
print(final_br_census_df.shape)
print(wands_br_lsoa_df.shape)
print(wands_year_census_df.shape)
print(no_exact_match_br_census_df_match.shape)
print(exact_match_br_census_df1.shape)
t = final_br_census_df.groupby(['year', 'CompanyID']).count()[['postcode']].sort_values(by = 'postcode').reset_index()
t = t[t['postcode'] > 1].copy().drop(columns = 'postcode')
print(t.shape)
t['duplicate'] = 'True'
tmp = final_br_census_df.set_index(['year', 'CompanyID']).join(t.set_index(['year', 'CompanyID'])
                                                              ).reset_index()
print(tmp[(tmp['duplicate'].apply(pd.isna))].shape)
print(tmp[(~tmp['duplicate'].apply(pd.isna))].shape)
print(tmp[(~tmp['duplicate'].apply(pd.isna)) & (tmp['source'] == 'Both')].shape)
tmp1 = tmp[tmp['duplicate'].apply(pd.isna)].copy().drop(columns = 'duplicate')
tmp2 = tmp[(~tmp['duplicate'].apply(pd.isna)) & (tmp['source'] == 'Both')].copy().drop(columns = 'duplicate')
final_br_census_df = pd.concat([tmp1, tmp2], ignore_index = True).sort_values(by = ['CompanyID', 'year'])
print(final_br_census_df.shape)
final_br_census_df.to_pickle("lam_south_final_br_census_df_v5.pkl")